# Benchmarking dos modelos


> A avaliação em modelos de detecção de objetos é fundamental para comparar e avaliar o desempenho destes. Permitindo selecionar o melhor modelo para uma tarefa, melhorar abordagens existentes e estabelecer padrões de referência. Além disso, o benchmarking auxilia na compreensão das capacidades e limitações de um modelo, impulsionando a inovação e validação experimental. Ao medir métricas quantitativas e identificar lacunas, o benchmark contribui para o avanço contínuo, promovendo eficiência e confiabilidade em cenários diversos.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Bibliotecas

In [2]:
%%capture
!pip install opendatasets
!pip install ultralytics
!pip install globox

In [3]:
import os
import cv2
import glob
import torch
import random
import ultralytics
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from ultralytics import YOLO
from globox import AnnotationSet, COCOEvaluator

### Dataset

In [4]:
import opendatasets as od

od.download('https://www.kaggle.com/datasets/lakshaytyagi01/fruit-detection')
#marcosvmedeiros
#d356a300e335f97bbf3559a47bb0d29f

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: marcosvmedeiros
Your Kaggle Key: ··········


100%|██████████| 501M/501M [00:06<00:00, 78.1MB/s]


In [5]:
DATA_DIR = '/content/fruit-detection/Fruits-detection'
data_valid = DATA_DIR+'/valid/images'
data_test = DATA_DIR+'/test/images'

### Dados de treinamento

In [6]:
data_train = AnnotationSet.from_yolo_v5(
    folder=DATA_DIR+"/train/labels/",
    image_folder=DATA_DIR+"/train/images/"
)

data_train.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │   1564 │  6070 │
│ 1     │   1205 │  2971 │
│ 2     │   1453 │  6027 │
│ 3     │   1818 │ 13938 │
│ 4     │    601 │  1372 │
│ 5     │    746 │  1683 │
├───────┼────────┼───────┤
│ Total │   7108 │ 32061 │
└───────┴────────┴───────┘

### Dados de validação

In [7]:
val_gts = AnnotationSet.from_yolo_v5(
    folder=DATA_DIR+"/valid/labels/",
    image_folder=DATA_DIR+"/valid/images/"
)

val_gts.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    188 │   557 │
│ 1     │    167 │   391 │
│ 2     │    199 │   809 │
│ 3     │    197 │  1100 │
│ 4     │     77 │   154 │
│ 5     │    107 │   217 │
├───────┼────────┼───────┤
│ Total │    914 │  3228 │
└───────┴────────┴───────┘

### Dados de teste

In [8]:
test_gts = AnnotationSet.from_yolo_v5(
    folder=DATA_DIR+"/test/labels/",
    image_folder=DATA_DIR+"/test/images/"
)

test_gts.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    113 │   422 │
│ 1     │     88 │   174 │
│ 2     │     94 │   366 │
│ 3     │     92 │   511 │
│ 4     │     40 │    87 │
│ 5     │     47 │    76 │
├───────┼────────┼───────┤
│ Total │    457 │  1636 │
└───────┴────────┴───────┘

## Benchmarking - Detecções dos modelos

## Inferência Yolov8 - NANO

In [9]:
model_nano = "/content/drive/MyDrive/TCC/Pesos/Yolov8/yolov8n/weights/best.pt"

In [10]:
%%capture
!yolo task=detect \
mode=predict model=$model_nano \
source=$data_valid\
save_txt=True save_conf=True name='Nano_valid'

In [11]:
%%capture
!yolo task=detect \
mode=predict model=$model_nano \
source=$data_test\
save_txt=True save_conf=True name='Nano_test'

### Detecções - Valid

In [12]:
dets_nano_v = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Nano_valid/labels/",
    image_folder="./runs/detect/Nano_valid/"
)

dets_nano_v.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    172 │   284 │
│ 1     │    149 │   213 │
│ 2     │    193 │   380 │
│ 3     │    201 │   535 │
│ 4     │     73 │   100 │
│ 5     │    105 │   189 │
├───────┼────────┼───────┤
│ Total │    825 │  1701 │
└───────┴────────┴───────┘

#### Métricas do modelo



In [13]:
evaluator = COCOEvaluator(
    ground_truths=val_gts,
    predictions=dets_nano_v
)

ap = evaluator.ap()
ar_100 = evaluator.ar_100()
ap_75 = evaluator.ap_75()
ap_small = evaluator.ap_small()

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃       ┃     AP ┃        ┃        ┃       ┃        ┃        ┃        ┃        ┃        ┃        ┃        ┃       ┃
┃ Label ┃  50:95 ┃  AP 50 ┃  AP 75 ┃  AP S ┃   AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃   AR S ┃   AR M ┃  AR L ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │ 20.67% │ 29.13% │ 22.28% │ 1.72% │ 13.08% │ 30.41% │ 13.49% │ 20.33% │ 20.67% │  2.59% │ 15.39% │ 39.9… │
│ 1     │ 18.52% │ 31.99% │ 17.89% │ 0.00% │  5.08% │ 23.34% │ 15.23% │ 18.52% │ 18.52% │  0.00% │  8.21% │ 30.5… │
│ 2     │ 19.16% │ 28.17% │ 20.07% │ 1.14% │  5.23% │ 37.58% │ 13.05% │ 19.16% │ 19.16% │  1.07% │  9.03% │ 44.6… │
│ 3     │ 16.27% │ 25.05% │ 17.35% │ 8.14% │ 11.68% │ 25.00% │  7.43% │ 14.17% │ 16.27% │ 13.67% │ 17.25% │ 34.2… │
│ 4     │ 21.61% │ 33.02% │ 21.30% │ 0.00% │  0.99% │ 27.44% │ 19.44% │ 21.61% │ 21.61% │  0.00% │  4.24% │ 34.0… │
│ 5     │ 30.93% │ 42.46% │ 33.44% │ 0.00% │  8.64% │ 36.59% │ 24.29% │ 30.93% │ 30.93% │  0.00% │ 17.18% │ 44.0… │
├───────┼────────┼────────┼────────┼───────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┤
│ Total │ 21.19% │ 31.64% │ 22.06% │ 1.83% │  7.45% │ 30.06% │ 18.58% │ 26.53% │ 27.18% │  2.89% │ 11.88% │ 37.9… │
└───────┴────────┴────────┴────────┴───────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┘

### Detecções - Test

In [14]:
dets_nano_t = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Nano_test/labels/",
    image_folder="./runs/detect/Nano_test/"
)

dets_nano_t.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    103 │   168 │
│ 1     │     71 │   102 │
│ 2     │     92 │   214 │
│ 3     │     91 │   225 │
│ 4     │     33 │    43 │
│ 5     │     54 │    82 │
├───────┼────────┼───────┤
│ Total │    405 │   834 │
└───────┴────────┴───────┘

#### Métricas do modelo

In [15]:
evaluator = COCOEvaluator(
    ground_truths=test_gts,
    predictions=dets_nano_t
)

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃       ┃      AP ┃        ┃        ┃       ┃       ┃        ┃        ┃        ┃        ┃       ┃        ┃        ┃
┃ Label ┃   50:95 ┃  AP 50 ┃  AP 75 ┃  AP S ┃  AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃   AR L ┃
┡━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ 0     │  17.10% │ 22.93% │ 19.06% │ 2.67% │ 7.69% │ 27.32% │ 13.35% │ 16.84% │ 17.10% │ 2.87% │ 10.00% │ 33.35% │
│ 1     │  17.89% │ 30.11% │ 16.32% │ 0.00% │ 0.41% │ 23.94% │ 16.97% │ 17.89% │ 17.89% │ 0.00% │  2.22% │ 29.64% │
│ 2     │  22.56% │ 33.62% │ 24.54% │ 0.00% │ 8.50% │ 41.38% │ 13.79% │ 22.50% │ 22.56% │ 0.00% │ 11.29% │ 49.47% │
│ 3     │  13.18% │ 20.23% │ 14.12% │ 5.39% │ 6.51% │ 27.17% │  7.91% │ 12.75% │ 13.18% │ 8.75% │  9.57% │ 35.58% │
│ 4     │  13.24% │ 20.95% │ 15.13% │ 0.00% │ 0.00% │ 21.08% │ 12.32% │ 13.24% │ 13.24% │ 0.00% │  0.00% │ 26.48% │
│ 5     │  34.36% │ 44.97% │ 37.02% │ 0.00% │ 5.35% │ 40.88% │ 29.76% │ 34.36% │ 34.36% │ 0.00% │ 10.00% │ 51.08% │
├───────┼─────────┼────────┼────────┼───────┼───────┼────────┼────────┼────────┼────────┼───────┼────────┼────────┤
│ Total │  19.72% │ 28.80% │ 21.03% │ 1.34% │ 4.74% │ 30.30% │ 18.13% │ 25.05% │ 25.29% │ 1.94% │  7.18% │ 37.60% │
└───────┴─────────┴────────┴────────┴───────┴───────┴────────┴────────┴────────┴────────┴───────┴────────┴────────┘

## Inferência Yolov8 - Medium

In [16]:
model_medium = "/content/drive/MyDrive/TCC/Pesos/Yolov8/yolov8m/weights/best.pt"

In [17]:
%%capture
!yolo task=detect \
mode=predict model=$model_medium \
source=$data_valid\
save_txt=True save_conf=True name='Medium_valid'

In [18]:
%%capture
!yolo task=detect \
mode=predict model=$model_medium \
source=$data_test\
save_txt=True save_conf=True name='Medium_test'

### Detecções - Valid

In [19]:
dets_medium_v = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Medium_valid/labels/",
    image_folder="./runs/detect/Medium_valid/"
)

dets_medium_v.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    162 │   268 │
│ 1     │    157 │   236 │
│ 2     │    201 │   378 │
│ 3     │    191 │   604 │
│ 4     │     61 │    74 │
│ 5     │    136 │   195 │
├───────┼────────┼───────┤
│ Total │    826 │  1755 │
└───────┴────────┴───────┘

#### Métricas do modelo



In [20]:
evaluator = COCOEvaluator(
    ground_truths=val_gts,
    predictions=dets_medium_v
)

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃       ┃     AP ┃        ┃        ┃       ┃        ┃        ┃        ┃        ┃        ┃        ┃        ┃       ┃
┃ Label ┃  50:95 ┃  AP 50 ┃  AP 75 ┃  AP S ┃   AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃   AR S ┃   AR M ┃  AR L ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │ 23.45% │ 31.01% │ 25.79% │ 0.66% │ 15.98% │ 34.00% │ 15.37% │ 23.45% │ 23.45% │  1.06% │ 18.48% │ 41.9… │
│ 1     │ 22.70% │ 36.47% │ 22.31% │ 0.00% │  4.98% │ 28.77% │ 18.44% │ 22.70% │ 22.70% │  0.00% │  8.33% │ 35.6… │
│ 2     │ 21.74% │ 29.14% │ 23.61% │ 0.17% │  7.36% │ 42.14% │ 15.12% │ 21.74% │ 21.74% │  0.33% │ 11.77% │ 49.1… │
│ 3     │ 17.84% │ 26.13% │ 19.34% │ 7.43% │ 13.04% │ 28.43% │  7.28% │ 15.83% │ 17.84% │ 13.85% │ 19.80% │ 38.8… │
│ 4     │ 23.26% │ 32.88% │ 24.85% │ 0.00% │  0.00% │ 29.73% │ 20.56% │ 23.26% │ 23.26% │  0.00% │  0.00% │ 34.8… │
│ 5     │ 38.29% │ 50.20% │ 42.96% │ 0.00% │ 29.53% │ 42.52% │ 30.83% │ 38.29% │ 38.29% │  0.00% │ 35.64% │ 49.4… │
├───────┼────────┼────────┼────────┼───────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┤
│ Total │ 24.55% │ 34.31% │ 26.48% │ 1.38% │ 11.81% │ 34.27% │ 20.74% │ 29.71% │ 30.27% │  2.54% │ 15.67% │ 41.6… │
└───────┴────────┴────────┴────────┴───────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┘

### Detecções - Test

In [21]:
dets_medium_t = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Medium_test/labels/",
    image_folder="./runs/detect/Medium_test/"
)

dets_medium_t.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    107 │   185 │
│ 1     │     83 │   111 │
│ 2     │     98 │   195 │
│ 3     │    100 │   271 │
│ 4     │     27 │    35 │
│ 5     │     55 │    76 │
├───────┼────────┼───────┤
│ Total │    417 │   873 │
└───────┴────────┴───────┘

#### Métricas do modelo

In [22]:
evaluator = COCOEvaluator(
    ground_truths=test_gts,
    predictions=dets_medium_t
)

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃       ┃     AP ┃        ┃        ┃       ┃        ┃        ┃        ┃        ┃        ┃       ┃        ┃        ┃
┃ Label ┃  50:95 ┃  AP 50 ┃  AP 75 ┃  AP S ┃   AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃   AR L ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ 0     │ 18.76% │ 24.60% │ 20.94% │ 1.91% │ 10.19% │ 29.89% │ 13.63% │ 18.55% │ 18.76% │ 1.91% │ 12.97% │ 36.74% │
│ 1     │ 19.31% │ 30.53% │ 19.68% │ 0.00% │  0.63% │ 24.42% │ 18.00% │ 19.31% │ 19.31% │ 0.00% │  2.22% │ 32.99% │
│ 2     │ 23.44% │ 34.51% │ 24.42% │ 0.00% │  7.75% │ 44.09% │ 15.63% │ 23.44% │ 23.44% │ 0.00% │  9.64% │ 51.93% │
│ 3     │ 15.36% │ 22.68% │ 16.91% │ 2.75% │  9.36% │ 29.71% │  8.75% │ 14.95% │ 15.36% │ 8.75% │ 12.51% │ 38.78% │
│ 4     │ 16.30% │ 22.88% │ 17.12% │ 0.00% │  3.56% │ 24.81% │ 14.10% │ 16.30% │ 16.30% │ 0.00% │  2.90% │ 30.93% │
│ 5     │ 41.23% │ 50.51% │ 43.01% │ 0.00% │ 10.69% │ 47.33% │ 36.09% │ 41.23% │ 41.23% │ 0.00% │ 10.00% │ 53.38% │
├───────┼────────┼────────┼────────┼───────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼────────┤
│ Total │ 22.40% │ 30.95% │ 23.68% │ 0.78% │  7.03% │ 33.37% │ 20.37% │ 27.42% │ 27.64% │ 1.78% │  8.37% │ 40.79% │
└───────┴────────┴────────┴────────┴───────┴────────┴────────┴────────┴────────┴────────┴───────┴────────┴────────┘

## Inferência Yolov8 - Large

In [24]:
model_large = "/content/drive/MyDrive/TCC/Pesos/Yolov8/yolov8l/weights/best.pt"

In [25]:
%%capture
!yolo task=detect \
mode=predict model=$model_large \
source=$data_valid\
save_txt=True save_conf=True name='Large_valid'

In [31]:
%%capture
!yolo task=detect \
mode=predict model=$model_large \
source=$data_test\
save_txt=True save_conf=True name='Large_teste'

### Detecções - Valid

In [32]:
dets_large_v = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Large_valid/labels/",
    image_folder="./runs/detect/Large_valid/"
)

dets_large_v.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    181 │   316 │
│ 1     │    153 │   221 │
│ 2     │    207 │   411 │
│ 3     │    214 │   603 │
│ 4     │     64 │    79 │
│ 5     │    126 │   214 │
├───────┼────────┼───────┤
│ Total │    841 │  1844 │
└───────┴────────┴───────┘

#### Métricas do modelo



In [33]:
evaluator = COCOEvaluator(
    ground_truths=val_gts,
    predictions=dets_large_v
)

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃       ┃     AP ┃        ┃        ┃        ┃        ┃        ┃        ┃        ┃        ┃       ┃        ┃       ┃
┃ Label ┃  50:95 ┃  AP 50 ┃  AP 75 ┃   AP S ┃   AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃  AR L ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │ 24.04% │ 31.89% │ 27.07% │  0.50% │ 13.31% │ 36.85% │ 15.35% │ 23.97% │ 24.04% │ 0.59% │ 15.76% │ 48.1… │
│ 1     │ 20.73% │ 34.99% │ 20.03% │  0.00% │  6.33% │ 26.05% │ 16.66% │ 20.73% │ 20.73% │ 0.00% │  7.95% │ 34.6… │
│ 2     │ 21.25% │ 29.19% │ 22.85% │  0.69% │  5.39% │ 42.27% │ 14.74% │ 21.25% │ 21.25% │ 0.33% │  9.91% │ 48.9… │
│ 3     │ 20.00% │ 29.07% │ 21.14% │ 10.34% │ 15.55% │ 30.80% │  8.52% │ 17.83% │ 20.00% │ 13.5… │ 21.81% │ 40.1… │
│ 4     │ 18.90% │ 28.98% │ 20.38% │  0.00% │  3.09% │ 23.59% │ 16.89% │ 18.90% │ 18.90% │ 0.00% │  3.64% │ 30.0… │
│ 5     │ 32.02% │ 41.83% │ 36.06% │  0.00% │ 29.04% │ 34.48% │ 24.35% │ 32.02% │ 32.02% │ 0.00% │ 36.15% │ 45.4… │
├───────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼───────┤
│ Total │ 22.82% │ 32.66% │ 24.59% │  1.92% │ 12.12% │ 32.34% │ 19.47% │ 29.05% │ 29.64% │ 2.41% │ 15.87% │ 41.2… │
└───────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┴───────┴────────┴───────┘

### Detecções - Test

In [34]:
dets_large_t = AnnotationSet.from_yolo_v5(
    folder="./runs/detect/Large_teste/labels/",
    image_folder="./runs/detect/Large_teste/"
)

dets_large_t.show_stats()

      Database Stats      
┏━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Label ┃ Images ┃ Boxes ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0     │    114 │   206 │
│ 1     │     81 │   110 │
│ 2     │    104 │   222 │
│ 3     │     92 │   246 │
│ 4     │     29 │    36 │
│ 5     │     50 │    94 │
├───────┼────────┼───────┤
│ Total │    422 │   914 │
└───────┴────────┴───────┘

#### Métricas do modelo

In [35]:
evaluator = COCOEvaluator(
    ground_truths=test_gts,
    predictions=dets_large_t
)

evaluator.show_summary()

                                                  COCO Evaluation                                                  
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃       ┃     AP ┃        ┃        ┃       ┃        ┃        ┃        ┃        ┃        ┃       ┃        ┃        ┃
┃ Label ┃  50:95 ┃  AP 50 ┃  AP 75 ┃  AP S ┃   AP M ┃   AP L ┃   AR 1 ┃  AR 10 ┃ AR 100 ┃  AR S ┃   AR M ┃   AR L ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ 0     │ 20.50% │ 26.26% │ 23.57% │ 1.19% │ 10.89% │ 32.79% │ 14.03% │ 20.31% │ 20.50% │ 0.64% │ 16.73% │ 40.35% │
│ 1     │ 17.71% │ 29.47% │ 16.16% │ 0.00% │  0.59% │ 22.33% │ 16.66% │ 17.71% │ 17.71% │ 0.00% │  0.56% │ 30.29% │
│ 2     │ 25.14% │ 35.42% │ 26.91% │ 0.00% │  8.99% │ 46.43% │ 15.31% │ 25.14% │ 25.14% │ 0.00% │ 12.21% │ 54.56% │
│ 3     │ 15.64% │ 22.51% │ 17.01% │ 2.95% │  9.83% │ 29.41% │  8.82% │ 14.94% │ 15.64% │ 8.00% │ 12.58% │ 36.74% │
│ 4     │ 12.12% │ 15.91% │ 14.29% │ 0.00% │  1.39% │ 18.82% │ 11.77% │ 12.12% │ 12.12% │ 0.00% │  2.26% │ 24.81% │
│ 5     │ 38.91% │ 47.55% │ 40.84% │ 0.00% │  5.35% │ 45.84% │ 36.09% │ 38.91% │ 38.91% │ 0.00% │ 10.00% │ 61.54% │
├───────┼────────┼────────┼────────┼───────┼────────┼────────┼────────┼────────┼────────┼───────┼────────┼────────┤
│ Total │ 21.67% │ 29.52% │ 23.13% │ 0.69% │  6.17% │ 32.61% │ 19.88% │ 28.16% │ 28.39% │ 1.44% │  9.06% │ 41.38% │
└───────┴────────┴────────┴────────┴───────┴────────┴────────┴────────┴────────┴────────┴───────┴────────┴────────┘